In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps
import re #needed for pattern matching


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Simple Example')


image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())



app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                            height = 250, width = 251))], href = 'https:www.snhu.edu', target = "_blank"),
    html.Center(html.B(html.H1('JKremhelmer \ CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        

    dcc.RadioItems(
    id = 'filter-type',
    option = [
        {'label' : 'All', 'value' :'All'},
        {'label':'Water Rescue', 'value':'Water'},
        {'label':'Mountain or Wilderness Rescue', 'value': 'Mountain'},
        {'label':'Disaster Rescue or Individual Tracking', 'value':'Disaster'},
    ],
    value='All'
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         editable=True,
                         row_selectable='single', #allows a row to be selected
                         selected_rows=[],
                         filter_action="native", # allows filter
                         sort_action="native", #allows sorting
                         page_action="native", #enablesd pagination
                         page_current=0, #set start page
                         page_size=10, #sets rows per pagge
                    

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
)

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'All':
        df = pd.DataFrame.from_records(shelter.getRecordsCriteria({}))
    elif filter_type == 'Water':
    
    labRegex = re.compile(".*lab.*", re.IGNORECASE)
    chesaRegex = re.compile(".*chesa.*", re.IGNORECASE)
    newRegex = re.compile(".newf.", re.IGNORECASE)
    
    df = pd.DataFrame.from_records(shelter.getRecordsCriteria({
        '$or':[#Regex isn't allowed in an $in helper so used $or
        {"breed": {'$regex': newRegex}}, #pass the regex filter
        {"breed": {'$regex':chesaRegex}},
        {"breed": {'$regex':labRegex}},
        ],
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
    }))
    elif filter_type == 'Mountain':
    
    germanRegex = re.compile(".*german.*", re.IGNOECASE)
    alaskanRegex = re.compile(".*mala.*", re.IGNORECASE)
    oldRegex = re.compile(".*old english.*", re.IGNORECASE)
    huskyRegex = re.compile(".*husk.*", re.IGNORECASE)
    rottRegex = re.compile(".*rott.*", re.IGNORECASE)
    
    df = pd.DataFrame.from_records(shelter.getRecordCriteria({
        '$or':[
        {"breed": {'$regex': germanRegex}},
        {"breed": {'$regex':alaskanRegex}},
        {"breed": {'$regex':oldRegex}},
        {"breed": {'$regex': huskyRegex}}, 
        {"breed": {'$regex':rottRegex}},
        ],
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
    }))
    elif filter_type == 'Disaster':
    
    germanRegex = re.compile(".*german.*", re.IGNOECASE)
    goldenRegex = re.compile(".*golden.*", re.IGNORECASE)
    bloodRegex = re.compile(".*blood english.*", re.IGNORECASE)
    doberRegex = re.compile(".*dober.*", re.IGNORECASE)
    rottRegex = re.compile(".*rott.*", re.IGNORECASE)
    
    df = pd.DataFrame.from_records(shelter.getRecordCriteria({
        '$or':[
        {"breed": {'$regex': germanRegex}},
        {"breed": {'$regex':goldenRegex}},
        {"breed": {'$regex':bloodRegex}},
        {"breed": {'$regex': doberRegex}}, 
        {"breed": {'$regex':rottRegex}},
        ],
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
    }))
    else:
        raise Exception("Unknown filter")
    
    columns=[{"name":i, "id": i, "deletable": False, "selectable": True} fro i in df.columns]
    data=df.to_dict('records')
    
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', selected_columns)]
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i},
        'background_color': '#D2F3FF'
    }for i in selected_columns]
    



# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    
    dffPie = pd.DataFrame.from_dict(viewData)
    
    return[
        dcc.Graphic(
            figure = px.pie(dffPie, names='breed')
        )
    ]

    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


SyntaxError: invalid syntax (4036489894.py, line 129)